<a href="https://colab.research.google.com/github/yardsale8/probability_simulations_in_R/blob/main/3_2_the_multinomial_distribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library(tidyverse)
library(devtools)
install_github('yardsale8/purrrfect', force = TRUE)
library(purrrfect)

# The Multinomial Distribution

In this notebook, we will look at simulations related to the multinomial distribution, specifically using sampling to generate raw outcomes and helper functions to count the occurances of each category.

## Review - The Set-Up

Here is the motivating scenario for the multinomial distribution.  As you might have guessed by the name, the scenario is similar to that resulting in the binomial distribution, except we are dealing with more than 2 categories.

* **Categories.** A single observation consists of one of $k$ categories, denoted as $c_i$ for $i=1,..,k$.
* **Fixed proabilities.** The probability of seeing category $c_i$ on a single observation is fixed at $p_i$ for $i=1,..,k$.
* **Fixed sample size.** Each experiment involves taking a sample of $n$ observations.
* **Independence.** The outcomes in the sample must be independent.



## The Multinomial Distribution

If that we generate raw outcomes as discussed above, the number of times that each category occurs is related to the [multinomial distribution](https://en.wikipedia.org/wiki/Multinomial_distribution). Any respectable mathematical statistics textbook will contain a proof of the following result.

**Theorem.** If the random variables $Y_i$ represent the counts of each category, that is $Y_i$ is the number of times that $c_i$ is observed in the sample for $i=1,..,k$, then $(Y_1, ..., Y_k)\sim multinomial(n, p_1, ..., p_k)$

## Simulating Multinomial Random Variates via Sampling with Replacement

One way to simulate the multinomial distribution is by

1. Generating vectors of $n$ raw observations using `sample` with `replace=TRUE` and the correct `prob`, and then
2. Creating count columns for each unique category.

### Example - Sampling Grades

Over the course of 20 years, the grade distribution in Introduction to Physics at Lake Woebegone College is as follows

Grade     |  Probability
----------|-------------
A         | 0.23  
B         | 0.26  
C         | 0.21  
D         | 0.18  
F         | 0.12  

We take a sample of 5 former students (with replacement) and record the number of students at each grade level.

**Questions.**
1. What is the probability of exactly three As and two Bs?
2. Given a sample had at least one A, what is the probability of at least one B?
3. What is the mean and variance in the number of Cs?
4. What is the covariance between the number of As and number of Bs?

#### Step 1 - Set up the simulation, including a Boolean column for each probability question.

In [58]:
grades <- c('A', 'B', 'C', 'D', 'F')
probs <- c(0.23, 0.26, 0.21, 0.18, 0.12)
n <- 5
N <- 100000
(replicate(N, sample(grades, n, prob = probs, replace = TRUE), .as = raw.grades)
 %>% mutate(num_A = num_successes_int(raw.grades, 'A'),
            num_B = num_successes_int(raw.grades, 'B'),
            num_C = num_successes_int(raw.grades, 'C'),
            num_D = num_successes_int(raw.grades, 'D'),
            num_F = num_successes_int(raw.grades, 'F'),
           )
) -> five_students

five_students %>% head

.trial,raw.grades,num_A,num_B,num_C,num_D,num_F
<dbl>,<list>,<int>,<int>,<int>,<int>,<int>
1,"A, A, A, A, B",4,1,0,0,0
2,"A, C, A, A, D",3,0,1,1,0
3,"B, F, B, D, A",1,2,0,1,1
4,"D, B, B, C, C",0,2,2,1,0
5,"B, C, C, C, A",1,1,3,0,0
6,"A, D, F, F, A",2,0,0,1,2


#### 1. Probability of three As and two Bs.

In [59]:
(five_students
 %>% mutate(three_As_and_two_Bs = (num_A == 3) & (num_B == 2))
 %>% estimate_prob(three_As_and_two_Bs)
)

three_As_and_two_Bs
<dbl>
0.00846


#### 2. Given a sample had at least one A, what is the probability of at least one B?

In [63]:
# Approach 1 - Group on given
(five_students
 %>% mutate(at_least_one_A = num_A >=1,
            at_least_one_B = num_B >= 1,
            )
 %>% group_by(at_least_one_A)
 %>% estimate_prob(at_least_one_B)
)

at_least_one_A,at_least_one_B
<lgl>,<dbl>
FALSE,0.8731476
TRUE,0.7432884


In [64]:
# Approach 2 - Filter on given case
(five_students
 %>% mutate(at_least_one_A = num_A >=1,
            at_least_one_B = num_B >= 1,
            )
 %>% filter(at_least_one_A)
 %>% estimate_prob(at_least_one_B)
)

at_least_one_B
<dbl>
0.7432884


#### 3. Estimate the mean and variance of the number of As.

In [65]:
p_A <- probs[1]

(five_students
 %>% summarise(est_mu_A = mean(num_A),
               est_var_A = var(num_A),
              )
 %>% mutate(exact_mu_A = n*p_A,
            exact_var_A = n*p_A*(1-p_A)
           )
 %>% relocate(exact_mu_A, .after = est_mu_A)
)

est_mu_A,exact_mu_A,est_var_A,exact_var_A
<dbl>,<dbl>,<dbl>,<dbl>
1.15366,1.15,0.8901775,0.8855


#### 4. Estimate the $Cov(Y_A, Y_B)$.

In [66]:
p_A <- probs[1]
p_B <- probs[2]

(five_students
 %>% summarise(est_cov_A_B = cov(num_A, num_B))
 %>% mutate(exact_cov_A_B = -1*n*p_A*p_B)
)

est_cov_A_B,exact_cov_A_B
<dbl>,<dbl>
-0.3012796,-0.299


### Cleaning up the code

You might have noticed that the code for counting each of the categories is very repetative.  We can clean up this code using the `multicounts(.data, .col, .labels)` helper function from the `purrrfect` library, which will create count columns for all entries in `.labels`.

#### Cleaned up simulation from example 1

Notice that replacing the large `mutate` (lefts as a comment for clarity) with `multcounts` leads to short and cleaner code.

In [15]:
grades <- c('A', 'B', 'C', 'D', 'F')
probs <- c(0.23, 0.26, 0.21, 0.18, 0.12)
n <- 5
N <- 100000
(replicate(N, sample(grades, n, prob = probs, replace = TRUE), .as = raw.grades)
#  %>% mutate(num_A = num_successes_int(raw.grades, 'A'),
#             num_B = num_successes_int(raw.grades, 'B'),
#             num_C = num_successes_int(raw.grades, 'C'),
#             num_D = num_successes_int(raw.grades, 'D'),
#             num_F = num_successes_int(raw.grades, 'F'),
#            )
 %>% multicounts(raw.grades, grades) # Computes all count columns
 %>% mutate(three_As_and_two_Bs = (num_A == 3) & (num_B == 2),
            at_least_one_A = num_A >=1,
            at_least_one_B = num_B >= 1,
            )
) -> five_students

five_students %>% head

.trial,raw.grades,num_A,num_B,num_C,num_D,num_F,three_As_and_two_Bs,at_least_one_A,at_least_one_B
<dbl>,<list>,<int>,<int>,<int>,<int>,<int>,<lgl>,<lgl>,<lgl>
1,"A, D, C, A, B",2,1,1,1,0,FALSE,TRUE,TRUE
2,"B, C, F, C, B",0,2,2,0,1,FALSE,FALSE,TRUE
3,"C, D, B, C, A",1,1,2,1,0,FALSE,TRUE,TRUE
4,"A, C, B, B, F",1,2,1,0,1,FALSE,TRUE,TRUE
5,"C, C, C, C, A",1,0,4,0,0,FALSE,TRUE,FALSE
6,"A, B, C, A, B",2,2,1,0,0,FALSE,TRUE,TRUE


To illustrate this point, let's delete the comment and inspect the resulting code.

In [23]:
grades <- c('A', 'B', 'C', 'D', 'F')
probs <- c(0.23, 0.26, 0.21, 0.18, 0.12)
n <- 5
N <- 10000
(replicate(N, sample(grades, n, prob = probs, replace = TRUE), .as = raw.grades)
 %>% multicounts(raw.grades, grades) # Makes all count columns simultaneously
 %>% mutate(three_As_and_two_Bs = (num_A == 3) & (num_B == 2))
) -> five_students

five_students %>% head(3)

.trial,raw.grades,num_A,num_B,num_C,num_D,num_F,three_As_and_two_Bs
<dbl>,<list>,<int>,<int>,<int>,<int>,<int>,<lgl>
1,"C, A, B, A, A",3,1,1,0,0,FALSE
2,"C, B, A, A, A",3,1,1,0,0,FALSE
3,"C, D, D, A, A",2,0,1,2,0,FALSE


In [24]:
five_students %>% estimate_all_prob

three_As_and_two_Bs
<dbl>
0.0072


## Generating multinomial counts directly using `rmultinom`

Alternatively, we can use the base R function `rmultinom` to generate the counts directly.  Unfortunately, `rmultinom` returns a matrix, which interacts poorly with the `purrrfect` tools.

In [48]:
rmultinom(1, n, probs) %>% str

 int [1:5, 1] 1 1 2 0 1


Notice the extra 1?  This is an unneeded dimension (at least when generating one outcome).  We can fix this by passing the output through `as.integer`.

In [49]:
rmultinom(1, n, probs) %>% as.integer %>% str

 int [1:5] 1 0 1 3 0


Wrap this expression in `replicate` to generate many trials.

In [52]:
replicate(10, rmultinom(1, n, probs) %>% as.integer, .as = counts)

.trial,counts
<dbl>,<list>
1,"1, 1, 1, 0, 2"
2,"0, 3, 1, 0, 1"
3,"2, 0, 0, 1, 2"
4,"2, 2, 0, 0, 1"
5,"1, 3, 0, 0, 1"
6,"0, 2, 1, 2, 0"
7,"0, 3, 1, 1, 0"
8,"0, 2, 3, 0, 0"
9,"2, 0, 2, 0, 1"


In this case it is conveninent to use `.reshape = 'split'` to split the counts into separate columns.

In [54]:
replicate(10000, rmultinom(1, n, probs) %>% as.integer, .as = counts, .reshape = 'split') -> from_r_split

from_r_split %>% head

.trial,counts.1,counts.2,counts.3,counts.4,counts.5
<dbl>,<int>,<int>,<int>,<int>,<int>
1,2,2,0,0,1
2,0,1,1,1,2
3,1,1,1,2,0
4,1,0,3,0,1
5,1,1,0,2,1
6,2,1,2,0,0


From this point, we can solve problems using the regular approach

In [57]:
( from_r_split
 %>% mutate(three_As_and_two_Bs = (counts.1 == 3) & (counts.2 == 2))
 %>% estimate_all_prob
)

three_As_and_two_Bs
<dbl>
0.0087
